# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [143]:
from xml.etree import ElementTree as ET
import numpy as np
import pandas as pd

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [134]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [135]:
# print names of all countries
for child in document_tree.getroot():
    print(child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [136]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print('* ' + element.find('name').text + ':',)
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print(capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [370]:
#read in the data and get the root element
document = ET.parse('./data/mondial_database.xml')
mondial = document.getroot()

In [371]:
#Q1 10 countries with the lowest infant mortality rates
all_records = [] #create a list of dictionary key/value pairs to load into a panda dataframe
for country in mondial.iter("country"):
    record = {}
    name = country.find("name")
    record["name"] = name.text
    infant_mortality = country.find("infant_mortality")
    if infant_mortality != None:
        record["infant_mortality"] = float(infant_mortality.text)
    else:
        record["infant_mortality"] = np.nan
        
    all_records.append(record)
    
df = pd.DataFrame(all_records)

In [372]:
#A1
df.sort_values(by = "infant_mortality")[0:10]

,infant_mortality,name
38,1.81,Monaco
98,2.13,Japan
117,2.48,Bermuda
36,2.48,Norway
106,2.53,Singapore
37,2.60,Sweden
10,2.63,Czech Republic
78,2.73,Hong Kong
79,3.13,Macao
44,3.15,Iceland


In [373]:
#Q2 10 cities with the largest population
all_records = [] #create a list of dictionary key/value pairs to load into a panda dataframe
cities = mondial.iter("city")
for city in cities:
    record = {}
    name = city.find("name") #get name of city. if city has two spellings find gets first one listed
    record["name"] = name.text
    pops = city.findall("population")
    for pop in pops:
        year = pop.get("year") #get the year the city's population is for
        record[year] = float(pop.text)
        
    all_records.append(record)

df = pd.DataFrame(all_records)

In [374]:
#A2.1
#We use 2011 to find the 10 largest cities by population
df[["2011","name"]].sort_values(by = "2011", ascending = False)[0:10]

,2011,name
1527,12442373.0,Mumbai
1582,11034555.0,Delhi
1515,8443675.0,Bangalore
1000,8250205.0,London
1382,8154051.0,Tehran
1470,7423137.0,Dhaka
1591,6731790.0,Hyderabad
1505,5577940.0,Ahmadabad
3056,5000000.0,Luanda
1556,4646732.0,Chennai


In [375]:
#Now we use the lastest year with an available population
#Create a new column with the population from the latest year available
df.head()

,1950,1957,1959,1960,1969,1970,1971,1972,1973,1976,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,name
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,418495.0,NaN,NaN,NaN,Tirana
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,77075.0,NaN,NaN,NaN,Shkodër
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,113249.0,NaN,NaN,NaN,Durrës
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,79513.0,NaN,NaN,NaN,Vlorë
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,78703.0,NaN,NaN,NaN,Elbasan


In [383]:
#Create new column called pop with the population from the latest year
#Fill in new column pop
pop = []
rows = df.iterrows()
for i,row in rows:
    years_with_pop = (row.iloc[0:-2].isnull() == False)
    if years_with_pop.any() == True:
        year = years_with_pop[years_with_pop == True].index[-1]
        pop.append(df.loc[i,year])
    else:
        pop.append(0)
        
df2 = df.assign(pop=pop)

In [387]:
#A2.2
#A2.1
#We use pop to find the 10 largest cities by population
df2[["pop","name"]].sort_values(by = "pop", ascending = False)[0:10]

,pop,name
1341,22315474.0,Shanghai
771,13710512.0,Istanbul
1527,12442373.0,Mumbai
479,11979529.0,Moskva
1340,11716620.0,Beijing
2810,11152344.0,São Paulo
1342,11090314.0,Tianjin
1064,11071424.0,Guangzhou
1582,11034555.0,Delhi
1067,10358381.0,Shenzhen


In [415]:
#Q3 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
all_records = [] #create a list of dictionary key/value pairs to load into a panda dataframe
countries = mondial.iter("country")
for country in countries:
    
    
    name = country.find("name").text
    pops = country.findall("population") #get all country population measurements
    pop = float(pops[-1].text) #assumes that populations are listed in order from earliest to latest
    year = int(pops[-1].get("year"))
    
    groups = country.findall("ethnicgroup") #get all ethnic groups
    for group in groups:
        record = {}
        
        record["name"] = name
        record["year"] = year
        record["pop"] = pop
        
        group_pop = float(group.get("percentage")) / 100 * pop
        group_name = group.text
        record["ethnicgroup"] = group_name
        record["ethnicpop"] = group_pop
        all_records.append(record)

df = pd.DataFrame(all_records)

In [438]:
#A3
vals = df.groupby("ethnicgroup")["ethnicpop"].sum()
vals.sort_values(ascending = False)[0:10]

ethnicgroup
Han Chinese    1.245059e+09
Indo-Aryan     8.718156e+08
European       4.948722e+08
African        3.183251e+08
Dravidian      3.027137e+08
Mestizo        1.577344e+08
Bengali        1.467769e+08
Russian        1.318570e+08
Japanese       1.265342e+08
Malay          1.219936e+08
Name: ethnicpop, dtype: float64

In [ ]:
#Q4 Name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [480]:
all_records = [] #create a list of dictionary key/value pairs to load into a panda dataframe
rivers = mondial.iter("river")
for river in rivers:
    record = {}
    
    cc = river.get("country")
    name = river.find("name").text
    if river.find("length") != None:
        length = float(river.find("length").text)
    else:
        length = 0
    
    record["country_code"] = cc
    record["length"] = length
    record["name"] = name
    
    all_records.append(record)

df = pd.DataFrame(all_records)

In [481]:
#A4a The longest river in the world is the Amazon river, which is primarily located in Brazil
df.sort_values(by = "length", ascending = False).iloc[0]

country_code    CO BR PE
length              6448
name            Amazonas
Name: 174, dtype: object

In [517]:
all_records = [] #create a list of dictionary key/value pairs to load into a panda dataframe
lakes = mondial.iter("lake")
for lake in lakes:
    record = {}
    
    cc = lake.get("country")
    name = lake.find("name").text
    if lake.find("area") != None:
        area = float(lake.find("area").text)
    else:
        area = 0
    
    record["country_code"] = cc
    record["area"] = area
    record["name"] = name
    
    all_records.append(record)

df = pd.DataFrame(all_records)

In [520]:
#A4b The largest lake in the world is the Caspian Sea, which is bordered by Russia and other countries
df.sort_values(by = "area", ascending = False).iloc[0]

area                    386400
country_code    R AZ KAZ IR TM
name               Caspian Sea
Name: 54, dtype: object

In [531]:
all_records = [] #create a list of dictionary key/value pairs to load into a panda dataframe
airports = mondial.iter("airport")
for airport in airports:
    record = {}
    
    cc = airport.get("country")
    name = airport.find("name").text
    if airport.find("elevation") != None:
        elevation = airport.find("elevation").text
    else:
        elevation = 0
    
    record["country_code"] = cc
    record["elevation"] = elevation
    record["name"] = name
    
    all_records.append(record)

df = pd.DataFrame(all_records)
df["elevation"] = df["elevation"].astype(float)

In [533]:
#A4c The highest airport in the world is El Alto International Airport at 4,063 meters and located in Bolivia
df.sort_values(by = "elevation", ascending = False).iloc[0]

country_code             BOL
elevation               4063
name            El Alto Intl
Name: 80, dtype: object